In [1]:
import os
from dotenv import load_dotenv
from langchain_community.utilities import SQLDatabase
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate

In [2]:
load_dotenv()
LANGSMITH_API_KEY = "LANGSMITH_API_KEY"
OPENAI_API_KEY = "OPENAI_API_KEY"

In [3]:
db = SQLDatabase.from_uri("sqlite:///sensor_data.db")

In [4]:
print(db.dialect)

sqlite


In [5]:
print(db.get_usable_table_names())

['sensor_readings']


In [6]:
db.run("SELECT * FROM sensor_readings LIMIT 10;")

'[(1, 22.5, 27.6, 2.2, 140, 13000, 3.1, 56, 1008, 14), (2, 19.1, 25.7, 3.6, 50, 15500, 2.8, 61, 1004, 12), (3, 24.8, 29.2, 1.8, 210, 16000, 3.6, 53, 1011, 17), (4, 18.7, 23.4, 0.9, 330, 17000, 3.0, 70, 1007, 10), (5, 23.2, 28.1, 2.5, 125, 14500, 2.7, 49, 1009, 15), (6, 17.5, 22.5, 3.1, 85, 14000, 2.6, 59, 1005, 11), (7, 20.0, 24.7, 1.2, 250, 13500, 2.5, 67, 1006, 13), (8, 21.3, 26.0, 2.0, 100, 15000, 3.4, 63, 1013, 12), (9, 25.1, 30.0, 2.9, 180, 20000, 4.2, 47, 1016, 19), (10, 18.2, 21.9, 1.0, 305, 12000, 2.2, 69, 1003, 8)]'

In [7]:
class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [8]:
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [9]:
system_message = """
Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

Only use the following tables:
{table_info}
"""

user_prompt = "Question: {input}"

query_prompt_template = ChatPromptTemplate(
    [("system", system_message), ("user", user_prompt)]
)

for message in query_prompt_template.messages:
    message.pretty_print()

================================ System Message ================================


Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

Only use the following tables:
{table_info}

================================ Human Message =================================

Question: {input}


In [10]:
from typing_extensions import Annotated


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [11]:
import certifi
import httpx
from openai import OpenAI

http_client = httpx.Client(verify=certifi.where())
client = OpenAI(api_key="OPENAI_API_KEY", http_client=http_client)

In [13]:
write_query({"question": "How many records are there?"})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}